In [2]:
import pycaret
import numpy
from http.server import SimpleHTTPRequestHandler, HTTPServer
from urllib import request, error
import pandas as pd
from pycaret.clustering import *
import urllib.parse
import os,sys
print("done...")

badwords = ['CREATE','TRUNCATE','truncate','DISTINCT','distinct','create','DELAY', 'group','ROWS', 'UPDATE', 'update', 'or', 'FETCH', 'INNER', 'NULL', 'SELECT', 'from','WAITFOR','waitfor', 'insert', 'PERCENT', 'between', 'like', 'null', 'select', 'and', 'FROM', 'BENCHMARK', 'SYSTEM', 'BETWEEN', 'benchmark', 'union', 'ALL', 'order', 'by', 'group' 'by', 'UID', 'concat', 'IN', 'OR', 'JOIN', 'delay', 'GROUP', 'BY', 'system', 'sum', 'LIKE', 'avg', 'having', 'uid', 'drop', 'chr', 'LIMIT', 'percent', 'limit', 'as', 'count', 'UNION', 'join', 'where', 'delete', 'WHERE', 'all', 'AVG', 'AND', 'COUNT', 'AS', 'rows', 'SLEEP', 'fetch', 'SUM', 'INSERT', 'in', 'HAVING', 'CONCAT', 'ORDER', 'BY', 'CHR', 'sleep', 'inner', 'DELETE', 'DROP']
python3 -m pip install --upgrade pip
def ExtractFeatures(path):
    path = urllib.parse.unquote(path)
    badwords_count=0
    single_q = path.count("'")
    double_q = path.count("\"")
    dash = path.count("--")+path.count("-")
    braces = path.count("(")+path.count(")")
    spaces = path.count(" ")
    
    for word in badwords:
        badwords_count += path.count(word)
    lst = [single_q,double_q,dashes,braces,spaces,badwords_count]
    print(lst)
    return pd.DataFrame([lst],columns = ['single_q','double_q','dashes','braces','spaces','badwords'])

ModuleNotFoundError: No module named 'pycaret'

In [ ]:
#Training Model
http = pd.read_csv(r'/home/asdf/Desktop/final_dat.csv')
clus1 = setup(data = http, normalize = True, numeric_features = ['single_q','double_q','dashes','braces','spaces','badwords'], ignore_features = ['method','body'])
kmeans = create_model('kmeans',num_cluster = 2)

In [ ]:
class SimpleHTTPProxy(SimpleHTTPRequestHandler):
    proxy_routes = {}

    @classmethod
    def set_routes(cls, proxy_routes):
        cls.proxy_routes = proxy_routes

    def do_GET(self):
    #print (self.proxy_routes)
    #print(self.path)
        parts = self.path.split("/")
        print (parts)
        live_data = ExtractFeatures(parts[3])
        result = predict_model(kmeans, data = live_data)
        print(result['Cluster'][0])
        if result['Cluster'][0] == "Cluster 1": 
            print("Intrusion Detected")
            #print (parts, Len(parts), parts[1])
        if len(parts) >= 2:
            #url self.proxy_routes[parts[2]]+'/'.join(parts[2:])
            #print(url)
            self.porxy_request('http://'+parts[2]+'/')
        else:
            super().do_GET()

    def porxy_request(self, url):
        #print ("prixy req')
        #exit()
        try:
            response = request.urlopen(url)
        except error.HTTPError as e:
            print ('err')
            self.send_response_only(e.code)
            self.end_headers()
            return
        self.send_response_only (response.status)
        for name, value in response.headers.items():
            self.send_header(name, value)
        self.end_headers()
        self.copyfile(response, self.wfile)
SimpleHTTPProxy.set_routes({'proxy_route': 'http://demo.testfire.net/'}) 
with HTTPServer(('127.0.0.1', 8080), SimpleHTTPProxy) as httpd:
    host, port = httpd.socket.getsockname()
    print (f'Listening on http://{host}:{port}')
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("\nkeyboard interrupt received, exiting.")